<a href="https://colab.research.google.com/github/bluevit/Recruit-Right-Precision-Hiring-with-AI-Insight/blob/master/org_NER_for_ResumeAnalysis_(3).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3>Training spaCy NER for Resume Analysis</h3>


In [1]:
!pip install numpy #==1.25

In [2]:
!pip install -U spacy #==3.7.2
!pip install spacy-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 758.8/758.8 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.2/314.2 kB 33.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 95.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

In [4]:
!pip install -U spacy
!python -m spacy download en_core_web_trf  # Download Transformer-based model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.9/237.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 735.6/735.6 kB 32.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
!pip install -U spacy-transformers
import spacy
from spacy.tokens import DocBin

In [6]:
# config_text = """[paths]
# train = "./train_data.spacy"
# dev = "./dev_data.spacy"
# vectors = null

# [nlp]
# lang = "en"
# pipeline = ["transformer", "ner"]

# [components]
# [components.ner]
# factory = "ner"

# [components.transformer]
# factory = "transformer"

# [training]
# dropout = 0.2
# patience = 5
# max_epochs = 20
# seed = 42
# gpu_allocator = "pytorch"
# accumulate_gradient = 1
# max_steps = 20000
# eval_frequency = 200
# frozen_components = []
# annotating_components = []
# dev_corpus = "corpora.dev"
# train_corpus = "corpora.train"
# before_to_disk = null
# before_update = null

# [training.logger]
# @loggers = "spacy.ConsoleLogger.v1"

# [training.batcher]
# @batchers = "spacy.batch_by_words.v1"
# discard_oversize = false
# tolerance = 0.2
# [training.batcher.size]
# @schedules = "compounding.v1"
# start = 100
# stop = 1000
# compound = 1.001

# [training.optimizer]
# @optimizers = "Adam.v1"
# learn_rate = 0.00005

# [training.score_weights]
# """

# with open("config.cfg", "w") as f:
#     f.write(config_text)

# print("✅ Final corrected config.cfg written.")


import json

# --- config.cfg content ---
config_text = """[paths]
train = "./train_data.spacy"
dev = "./dev_data.spacy"
vectors = null

[nlp]
lang = "en"
pipeline = ["transformer", "ner"]

[components]
[components.ner]
factory = "ner"

[components.transformer]
factory = "transformer"

[training]
dropout = 0.2
patience = 5
max_epochs = 20
seed = 42
gpu_allocator = "pytorch"
accumulate_gradient = 1
max_steps = 20000
eval_frequency = 200
frozen_components = []
annotating_components = []
dev_corpus = "corpora.dev"
train_corpus = "corpora.train"
before_to_disk = null
before_update = null

[training.logger]
@loggers = "spacy.ConsoleLogger.v1"

[training.batcher]
@batchers = "spacy.batch_by_words.v1"
discard_oversize = false
tolerance = 0.2
[training.batcher.size]
@schedules = "compounding.v1"
start = 100
stop = 1000
compound = 1.001

[training.optimizer]
@optimizers = "Adam.v1"
learn_rate = 0.00005

[training.score_weights]
"""

# --- JSON version of the same config ---
config_json = {
    "paths": {
        "train": "./train_data.spacy",
        "dev": "./dev_data.spacy",
        "vectors": None
    },
    "nlp": {
        "lang": "en",
        "pipeline": ["transformer", "ner"]
    },
    "components": {
        "ner": {"factory": "ner"},
        "transformer": {"factory": "transformer"}
    },
    "training": {
        "dropout": 0.2,
        "patience": 5,
        "max_epochs": 20,
        "seed": 42,
        "gpu_allocator": "pytorch",
        "accumulate_gradient": 1,
        "max_steps": 20000,
        "eval_frequency": 200,
        "frozen_components": [],
        "annotating_components": [],
        "dev_corpus": "corpora.dev",
        "train_corpus": "corpora.train",
        "before_to_disk": None,
        "before_update": None,
        "logger": {
            "@loggers": "spacy.ConsoleLogger.v1"
        },
        "batcher": {
            "@batchers": "spacy.batch_by_words.v1",
            "discard_oversize": False,
            "tolerance": 0.2,
            "size": {
                "@schedules": "compounding.v1",
                "start": 100,
                "stop": 1000,
                "compound": 1.001
            }
        },
        "optimizer": {
            "@optimizers": "Adam.v1",
            "learn_rate": 0.00005
        },
        "score_weights": {}
    }
}

# --- Write files ---
with open("config.cfg", "w") as cfg_file:
    cfg_file.write(config_text)

with open("config.json", "w") as json_file:
    json.dump(config_json, json_file, indent=2)

print("✅ Successfully written config.cfg and config.json.")

✅ Successfully written config.cfg and config.json.


In [7]:
# !cat config.cfg

In [8]:
!git clone https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git

# !git clone https://github.com/Mehyarmlaweh/NER-Annotated-CVs.git


Cloning into 'CV-Parsing-using-Spacy-3'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (76/76), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 82 (delta 14), reused 74 (delta 14), pack-reused 6 (from 1)
Receiving objects: 100% (82/82), 5.62 MiB | 10.69 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [9]:
import json
cv_data = json.load(open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json','r'))

In [10]:
import json
import spacy
from spacy.tokens import DocBin
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Load training data JSON again
with open('/content/CV-Parsing-using-Spacy-3/data/training/train_data.json', 'r', encoding='utf-8') as f:
    cv_data = json.load(f)

# Split into train/dev
train_data, dev_data = train_test_split(cv_data, test_size=0.1, random_state=42)


In [14]:
# # def convert_to_spacy(data, filename):
# #     nlp = spacy.blank("en")
# #     db = DocBin()
# #     skipped = 0

# #     for text, annot in tqdm(data):
# #         doc = nlp.make_doc(text)
# #         ents = []
# #         seen_tokens = set()
# #         valid = True

# #         for start, end, label in annot["entities"]:
# #             if start >= end or start < 0 or end > len(text):
# #                 valid = False
# #                 break

# #             span = doc.char_span(start, end, label=label, alignment_mode="contract")
# #             if span is None:
# #                 valid = False
# #                 break

# #             if any(tok.i in seen_tokens for tok in span):
# #                 valid = False
# #                 break

# #             ents.append(span)
# #             seen_tokens.update([tok.i for tok in span])

# #         if valid:
# #             try:
# #                 doc.ents = ents
# #                 db.add(doc)
# #             except Exception:
# #                 skipped += 1
# #         else:
# #             skipped += 1

# #     db.to_disk(filename)
# #     print(f"✅ Saved {len(data)-skipped} examples to {filename}. Skipped {skipped}.")


# def convert_to_spacy(data, filename):
#     import spacy
#     from spacy.tokens import DocBin
#     from tqdm import tqdm

#     nlp = spacy.blank("en")
#     db = DocBin()
#     skipped = 0

#     for text, annot in tqdm(data):
#         text = text.strip()  # clean whitespace
#         doc = nlp.make_doc(text)
#         ents = []
#         seen_tokens = set()
#         valid = True

#         for start, end, label in annot["entities"]:
#             if start >= end or start < 0 or end > len(text):
#                 valid = False
#                 break

#             span = doc.char_span(start, end, label=label, alignment_mode="contract")
#             if span is None or span.text.strip() != span.text:
#                 valid = False
#                 break

#             if any(tok.i in seen_tokens for tok in span):
#                 valid = False
#                 break

#             ents.append(span)
#             seen_tokens.update([tok.i for tok in span])

#         if valid:
#             try:
#                 doc.ents = ents
#                 db.add(doc)
#             except Exception:
#                 skipped += 1
#         else:
#             skipped += 1

#     db.to_disk(filename)
#     print(f"✅ Saved {len(data)-skipped} examples to {filename}. Skipped {skipped}.")


import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

def convert_to_spacy_and_extract(data, filename, model_path):
    nlp = spacy.blank("en")
    db = DocBin()
    skipped = 0

    print("📦 Converting training data to .spacy format...")
    for text, annot in tqdm(data):
        text = text.strip()  # Clean whitespace
        doc = nlp.make_doc(text)
        ents = []
        seen_tokens = set()
        valid = True

        for start, end, label in annot["entities"]:
            if start >= end or start < 0 or end > len(text):
                valid = False
                break

            span = doc.char_span(start, end, label=label, alignment_mode="contract")
            if span is None or span.text.strip() != span.text:
                valid = False
                break

            if any(tok.i in seen_tokens for tok in span):
                valid = False
                break

            ents.append(span)
            seen_tokens.update([tok.i for tok in span])

        if valid:
            try:
                doc.ents = ents
                db.add(doc)
            except Exception:
                skipped += 1
        else:
            skipped += 1

    db.to_disk(filename)
    print(f"✅ Saved {len(data) - skipped} examples to {filename}. Skipped {skipped}.")

    # Now load the trained model and extract entities from each sample
    if not model_path:
        raise ValueError("❌ Please provide the path to your trained NER model.")

    print("\n🔍 Extracting entities using trained model...\n")
    ner_model = spacy.load(model_path)

    for text, _ in data:
        print(f"📝 Text: {text.strip()}")
        doc = ner_model(text)
        for ent in doc.ents:
            print(f"{ent.text:<20} → {ent.label_}")
        print("-" * 40)


In [15]:
convert_to_spacy(train_data, "train_data.spacy")
convert_to_spacy(dev_data, "dev_data.spacy")


100%|██████████| 180/180 [00:01<00:00, 148.44it/s]


✅ Saved 81 examples to train_data.spacy. Skipped 99.


100%|██████████| 20/20 [00:00<00:00, 118.18it/s]

✅ Saved 8 examples to dev_data.spacy. Skipped 12.


In [ ]:
# len(cv_data)

In [ ]:
# you can see the data(json) by uncommenting below statement
#cv_data

In [16]:
#!python -m spacy train config.cfg --output ./output --gpu-id 0

!python -m spacy train config.cfg --output ./output --gpu-id 0

✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
tokenizer_config.json: 100% 25.0/25.0 [00:00<00:00, 174kB/s]
config.json: 100% 481/481 [00:00<00:00, 3.21MB/s]
vocab.json: 100% 899k/899k [00:00<00:00, 25.7MB/s]
merges.txt: 100% 456k/456k [00:00<00:00, 3.12MB/s]
tokenizer.json: 100% 1.36M/1.36M [00:00<00:00, 4.90MB/s]
2025-07-12 10:51:35.634135: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752317495.882519    3336 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752317495.952206    3336 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2

In [17]:
!python -m spacy init fill-config /content/CV-Parsing-using-Spacy-3/data/training/base_config.cfg /content/CV-Parsing-using-Spacy-3/data/training/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/CV-Parsing-using-Spacy-3/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


---
config.cfg must be modified in order to reduce the training time<br>
(generated in *content/CV-Parsing-using-Spacy-3/data/training/config.cfg*)<br><br>
max_steps parameter must be set to some low number like 2000 or 4000, which will drastically reduce the training time.</h3>
<hr>

This function takes a dataset of text and annotations, processes each text to identify and label entities, and stores the results in a DocBin object that can be used for further training or analysis with spaCy.

In [19]:
# def get_spacy_doc(file, data):
#   nlp = spacy.blank('en')
#   db = DocBin()
#   for text, annot in tqdm(data):
#     doc = nlp.make_doc(text)
#     annot = annot['entities']

#     ents = []
#     entity_indices = []

#     for start, end, label in annot:
#       skip_entity = False
#       for idx in range(start, end):
#         if idx in entity_indices:
#           skip_entity = True
#           break
#       if skip_entity == True:
#         continue
#       entity_indices = entity_indices + list(range(start, end))
#       try:
#         span = doc.char_span(start, end, label=label, alignment_mode='strict')
#       except:
#         continue

#       if span is None:
#         err_data = str([start, end]) + ' ' + str(text) + '\n'
#         file.write(err_data)
#       else :
#         ents.append(span)
#     try:
#       doc.ents = ents
#       db.add(doc)
#     except:
#       pass
#       #print(text)
#   return db

def get_spacy_doc(file, data):
    import spacy
    from spacy.tokens import DocBin
    from tqdm import tqdm

    nlp = spacy.blank('en')
    db = DocBin()

    for text, annot in tqdm(data):
        text = text.strip()
        doc = nlp.make_doc(text)
        ents = []
        entity_indices = set()

        for start, end, label in annot['entities']:
            # Skip overlapping spans
            if any(i in entity_indices for i in range(start, end)):
                continue

            try:
                span = doc.char_span(start, end, label=label, alignment_mode='strict')
            except:
                continue

            if span is None:
                file.write(f"Invalid span: [{start}, {end}], label: {label}, text: '{text[start:end]}'\n")
                continue

            ents.append(span)
            entity_indices.update(range(start, end))

        try:
            doc.ents = ents
            db.add(doc)
        except:
            pass

    return db


In [ ]:
# !pip install optuna
# import optuna

# def objective(trial):
#     dropout = trial.suggest_float("dropout", 0.1, 0.3)
#     batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])

#     # Update config.cfg
#     with open("/content/CV-Parsing-using-Spacy-3/data/training/config.cfg", "r") as file:
#         config = file.read()

#     config = config.replace("dropout = 0.2", f"dropout = {dropout}")
#     config = config.replace("batch_size = 32", f"batch_size = {batch_size}")

#     with open("/content/CV-Parsing-using-Spacy-3/data/training/config.cfg", "w") as file:
#         file.write(config)

#     # Train the model
#     !python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --gpu-id 0

#     # Check if the model was created
#     import os
#     if os.path.exists("./output/model-best"):  # Check if the model directory exists
#         # Load trained model and evaluate
#         nlp = spacy.load("./output/model-best")
#         test_text = "John Doe worked at Microsoft as a Data Scientist from 2019-2023."
#         doc = nlp(test_text)

#         return len(doc.ents)  # Optimize based on entity count
#     else:
#         print("Model not found at ./output/model-best. Check training output.")
#         return 0  # Return a default value if the model is not found

# study = optuna.create_study(direction="maximize")
# study.optimize(objective, n_trials=5)

# print("Best Hyperparameters:", study.best_params)

**Training**

In [20]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(cv_data, test_size=0.3)

In [21]:
file = open('error.txt', 'w')
db = get_spacy_doc(file, train)
db.to_disk('train_data.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_data.spacy')

100%|██████████| 60/60 [00:00<00:00, 123.93it/s]


In [22]:
!python -m spacy train /content/CV-Parsing-using-Spacy-3/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0

ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
2025-07-12 10:54:50.034463: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752317690.054402    4198 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752317690.060322    4198 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-12 10:54:50.082818: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate co

<h3>Testing the Trained Model</h3>

In [23]:
nlp = spacy.load('/content/output/model-best')

In [24]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 6.0 MB/s eta 0:00:00


In [25]:
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 46.6 MB/s eta 0:00:00


In [26]:
# from pdfminer.high_level import extract_text

# text = extract_text("/content/M_Abdullah_Resume.pdf")
# print(text)

In [ ]:
# doc = nlp(text)
# for ent in doc.ents:
#   print(ent.text, '---->' , ent.label_)

testing


In [27]:
import spacy

# Load your trained model
nlp = spacy.load("output/model-last")

In [ ]:
# # Read resume text
# from pdfminer.high_level import extract_text
# text = extract_text("/content/M_Abdullah_Resume.pdf")

In [ ]:
# # Process with NER model
# doc = nlp(text)
# for ent in doc.ents:
#     print(ent.text, "---->", ent.label_)

In [ ]:
# import spacy
# from spacy.matcher import Matcher
# import re
# from pdfminer.high_level import extract_text

# # Load the trained model
# nlp = spacy.load("output/model-last")

# # Extract text from the new resume
# text = extract_text("/content/Muhammad Fareed Full Stack Developer Resume - Muhammad Fareed.pdf")
# doc = nlp(text)

# # Store extracted entities in a dictionary to remove duplicates
# extracted_data = {"Name": set(), "Degree": set(), "College Name": set(), "Location": set(), "Skills": set()}

# # Extract NER results
# for ent in doc.ents:
#     extracted_data[ent.label_].add(ent.text)

# # Extract experience using regex
# exp_pattern = re.findall(r'(\d+)\+?\s*years?\s*of\s*experience', text, re.IGNORECASE)
# if exp_pattern:
#     extracted_data["Experience"] = {exp_pattern[0] + " years"}

# # Rule-based matching for skills
# matcher = Matcher(nlp.vocab)
# skills_patterns = [
#     [{"LOWER": "python"}],
#     [{"LOWER": "javascript"}],
#     [{"LOWER": "machine"}, {"LOWER": "learning"}],
#     [{"LOWER": "data"}, {"LOWER": "science"}],
#     [{"LOWER": "mern"}, {"LOWER": "stack"}],
#     [{"LOWER": "react"}],
#     [{"LOWER": "firebase"}],
#     [{"LOWER": "fast"}, {"LOWER": "api"}],
#     [{"LOWER": "solidity"}],
#     [{"LOWER": "postgresql"}]
# ]
# matcher.add("SKILLS", skills_patterns)

# matches = matcher(doc)
# for match_id, start, end in matches:
#     extracted_data["Skills"].add(doc[start:end].text)

# # Display the cleaned-up results
# print("---- Cleaned Entity Recognition Output ----")
# for key, values in extracted_data.items():
#     if values:
#         print(f"{key}: {', '.join(values)}")


In [ ]:
# import spacy
# from spacy.matcher import Matcher
# import re
# from pdfminer.high_level import extract_text

# # Load the trained model
# nlp = spacy.load("output/model-last")

# # Extract text from the new resume
# text = extract_text("/content/Muhammad Fareed Full Stack Developer Resume - Muhammad Fareed.pdf")
# doc = nlp(text)

# # Store extracted entities in a dictionary to remove duplicates
# extracted_data = {"Name": set(), "Degree": set(), "College Name": set(), "Location": set(), "Skills": set()}

# # Extract NER results
# for ent in doc.ents:
#     extracted_data[ent.label_].add(ent.text)

# # Extract experience using regex
# exp_pattern = re.findall(r'(\d+)\+?\s*years?\s*of\s*experience', text, re.IGNORECASE)
# if exp_pattern:
#     extracted_data["Experience"] = {exp_pattern[0] + " years"}

# # Rule-based matching for skills
# matcher = Matcher(nlp.vocab)
# skills_patterns = [
#     [{"LOWER": "python"}],
#     [{"LOWER": "javascript"}],
#     [{"LOWER": "machine"}, {"LOWER": "learning"}],
#     [{"LOWER": "data"}, {"LOWER": "science"}],
#     [{"LOWER": "mern"}, {"LOWER": "stack"}],
#     [{"LOWER": "react"}],
#     [{"LOWER": "firebase"}],
#     [{"LOWER": "fast"}, {"LOWER": "api"}],
#     [{"LOWER": "solidity"}],
#     [{"LOWER": "postgresql"}]
# ]
# matcher.add("SKILLS", skills_patterns)

# matches = matcher(doc)
# for match_id, start, end in matches:
#     extracted_data["Skills"].add(doc[start:end].text)

# # Display the cleaned-up results
# print("---- Cleaned Entity Recognition Output ----")
# for key, values in extracted_data.items():
#     if values:
#         print(f"{key}: {', '.join(values)}")


In [ ]:
# import spacy
# import re
# from pdfminer.high_level import extract_text
# from dateutil import parser
# from datetime import datetime

# # Load the trained NER model
# nlp = spacy.load("output/model-last")

# # Extract text from the resume
# text = extract_text("/content/Muhammad Fareed Full Stack Developer Resume - Muhammad Fareed.pdf")
# doc = nlp(text)

# # Store extracted entities in a dictionary
# extracted_data = {
#     "Name": set(),
#     "Degree": set(),
#     "College Name": set(),
#     "Location": set(),
#     "Skills": set(),
#     "Experience": set(),
# }

# # Extract Named Entities (NER)
# for ent in doc.ents:
#     extracted_data[ent.label_].add(ent.text)

# # Extract job experience details (using regex)
# experience_pattern = re.findall(
#     r"([\w\s\-/]+)\s*[\n]*\s*(\w+\s*\d{4})\s*–\s*(\w+\s*\d{4}|Present)", text
# )

# total_experience = 0
# for job_title, start_date, end_date in experience_pattern:
#     try:
#         start_date = parser.parse(start_date)
#         end_date = (
#             parser.parse(end_date) if "Present" not in end_date else datetime.today()
#         )
#         experience_months = (end_date.year - start_date.year) * 12 + (
#             end_date.month - start_date.month
#         )
#         total_experience += experience_months
#         extracted_data["Experience"].add(
#             f"{job_title.strip()} ({start_date.strftime('%b %Y')} - {end_date.strftime('%b %Y')})"
#         )
#     except Exception:
#         continue  # Skip parsing errors

# # Rule-based matching for skills
# matcher = Matcher(nlp.vocab)
# skills_patterns = [
#     [{"LOWER": "python"}],
#     [{"LOWER": "javascript"}],
#     [{"LOWER": "machine"}, {"LOWER": "learning"}],
#     [{"LOWER": "data"}, {"LOWER": "science"}],
#     [{"LOWER": "mern"}, {"LOWER": "stack"}],
#     [{"LOWER": "react"}],
#     [{"LOWER": "firebase"}],
#     [{"LOWER": "fast"}, {"LOWER": "api"}],
#     [{"LOWER": "solidity"}],
#     [{"LOWER": "postgresql"}]
# ]
# matcher.add("SKILLS", skills_patterns)

# matches = matcher(doc)
# for match_id, start, end in matches:
#     extracted_data["Skills"].add(doc[start:end].text)


# # Convert total experience to years & months
# if total_experience > 0:
#     years, months = divmod(total_experience, 12)
#     extracted_data["Total Experience"] = f"{years} years {months} months"

# # Display the cleaned-up results
# print("\n---- Extracted Information ----")
# for key, values in extracted_data.items():
#     if values:
#         print(f"{key}: {','.join(values)}")


In [30]:
# ------------------------------------------
# ✅ Resume Information Extractor (NER + Experience + Skills)
# ------------------------------------------

import spacy
import re
from pdfminer.high_level import extract_text
from dateutil import parser
from datetime import datetime
from spacy.matcher import Matcher

# Load the trained NER model
nlp = spacy.load("output/model-best")  # Ensure this path is correct

# Extract text from the resume PDF
text = extract_text("/content/Abdullah_CV.pdf")
doc = nlp(text)

# Store extracted entities dynamically
extracted_data = {
    "Name": set(),
    "College Name": set(),
    "Location": set(),
    "Skills": set(),
    # "Experience": set(),
}

# Extract NER entities
for ent in doc.ents:
    label = ent.label_
    if label not in extracted_data:
        extracted_data[label] = set()
    extracted_data[label].add(ent.text.strip())

# Step 1: Extract raw job experience using regex
experience_pattern = re.findall(
    r"(.+?)\s*\(?([A-Za-z]+\s+\d{2,4})\)?\s*[–-]\s*\(?([A-Za-z]+\s+\d{2,4}|Present)\)?",
    text
)

# Step 2: Extract and merge overlapping experience durations
raw_date_ranges = []
for job_title, start_str, end_str in experience_pattern:
    try:
        start_date = parser.parse(start_str, default=datetime(1900, 1, 1))
        end_date = datetime.today() if "Present" in end_str else parser.parse(end_str, default=datetime.today())

        raw_date_ranges.append((start_date, end_date))

        extracted_data["Experience"].add(
            f"{job_title.strip()} ({start_date.strftime('%b %Y')} - {end_date.strftime('%b %Y')})"
        )
    except:
        continue

# Step 3: Merge overlapping date ranges
raw_date_ranges.sort()
merged_ranges = []
for current_start, current_end in raw_date_ranges:
    if not merged_ranges:
        merged_ranges.append((current_start, current_end))
    else:
        last_start, last_end = merged_ranges[-1]
        if current_start <= last_end:
            merged_ranges[-1] = (last_start, max(last_end, current_end))
        else:
            merged_ranges.append((current_start, current_end))

# Step 4: Calculate total merged experience
total_months = sum((end.year - start.year) * 12 + (end.month - start.month) for start, end in merged_ranges)
years, months = divmod(total_months, 12)
extracted_data["Total Experience"] = {f"{years} years {months} months"}

# Step 5: Rule-based skill matching
matcher = Matcher(nlp.vocab)
skills = [
    "python", "javascript", "machine learning", "data science",
    "mern stack", "react", "firebase", "fast api", "solidity", "postgresql"
]
for skill in skills:
    pattern = [{"LOWER": token} for token in skill.split()]
    matcher.add("SKILLS", [pattern])

matches = matcher(doc)
for match_id, start, end in matches:
    extracted_data["Skills"].add(doc[start:end].text)

# Display extracted results
print("\n---- Extracted Resume Information ----")
for key, values in extracted_data.items():
    if values:
        print(f"{key}:\n  - " + "\n  - ".join(values))



---- Extracted Resume Information ----
Name:
  - MUHAMMAD ABDULLAH
College Name:
  - Hamdard University
  - Government Degree College SRE-Majeed, Karachi

2017/April
2019/July

PROJECTS

Matriculation - Science
-
Computer Science

Happy Palace Grammar School, Karachi

Machine Learning Object Detection

GitHub Link
The model is a real-time image classification system designed to predict the class of objects (e.g., fruits
and vegetables) captured through a live video feed from a webcam. It uses a pre-trained deep learning
model loaded from a .h5 file to classify images into predefined classes.

Power BI

HR Dashboard
A basic analysis of the employees of the company.

Machine Learning Stock Prediction

A basic prediction of stocks, using Linear Regression.

EXPERIENCE

GitHub Link

GitHub Link

Feb 2025 – Present Project Manager Intern

Binate Digital
Assisting in managing software development projects, coordinating with cross-functional teams, and en-
suring timely delivery of project m

In [38]:
# ------------------------------------------
# ✅ Resume Information Extractor (NER + Experience + Skills)
# ------------------------------------------

import spacy
import re
from pdfminer.high_level import extract_text
from dateutil import parser
from datetime import datetime
from spacy.matcher import Matcher

# Load the trained NER model
nlp = spacy.load("output/model-best")  # Ensure this path is correct

# Extract text from the resume PDF
text = extract_text("/content/Muhammad Fareed Full Stack Developer Resume - Muhammad Fareed.pdf")
doc = nlp(text)

# Store extracted entities dynamically
extracted_data = {
    "Name": set(),
    "College Name": set(),
    # "Location": set(),
    "Skills": set(),
    # "Experience": set(),
}

# Extract NER entities
for ent in doc.ents:
    label = ent.label_
    if label not in extracted_data:
        extracted_data[label] = set()
    extracted_data[label].add(ent.text.strip())

# Step 1: Extract raw job experience using regex
experience_pattern = re.findall(
    r"(.+?)\s*\(?([A-Za-z]+\s+\d{2,4})\)?\s*[–-]\s*\(?([A-Za-z]+\s+\d{2,4}|Present)\)?",
    text
)

# Step 2: Extract and merge overlapping experience durations
raw_date_ranges = []
for job_title, start_str, end_str in experience_pattern:
    try:
        start_date = parser.parse(start_str, default=datetime(1900, 1, 1))
        end_date = datetime.today() if "Present" in end_str else parser.parse(end_str, default=datetime.today())

        raw_date_ranges.append((start_date, end_date))

        extracted_data["Experience"].add(
            f"{job_title.strip()} ({start_date.strftime('%b %Y')} - {end_date.strftime('%b %Y')})"
        )
    except:
        continue

# Step 3: Merge overlapping date ranges
raw_date_ranges.sort()
merged_ranges = []
for current_start, current_end in raw_date_ranges:
    if not merged_ranges:
        merged_ranges.append((current_start, current_end))
    else:
        last_start, last_end = merged_ranges[-1]
        if current_start <= last_end:
            merged_ranges[-1] = (last_start, max(last_end, current_end))
        else:
            merged_ranges.append((current_start, current_end))

# Step 4: Calculate total merged experience
total_months = sum((end.year - start.year) * 12 + (end.month - start.month) for start, end in merged_ranges)
years, months = divmod(total_months, 12)
extracted_data["Total Experience"] = {f"{years} years {months} months"}

# Step 5: Rule-based skill matching
matcher = Matcher(nlp.vocab)
skills = [
    "python", "javascript", "machine learning", "data science",
    "mern stack", "react", "firebase", "fast api", "solidity", "postgresql"
]
for skill in skills:
    pattern = [{"LOWER": token} for token in skill.split()]
    matcher.add("SKILLS", [pattern])

matches = matcher(doc)
for match_id, start, end in matches:
    extracted_data["Skills"].add(doc[start:end].text)

# Display extracted results
print("\n---- Extracted Resume Information ----")
for key, values in extracted_data.items():
    if values:
        print(f"{key}:\n  - " + "\n  - ".join(values))



---- Extracted Resume Information ----
Name:
  - MUHAMMAD FAREED
College Name:
  - Hamdard University
Skills:
  - MERN stack
  - FAST API
  - Firebase
  - MERN Stack
  - React
  - PostgreSQL
  - Python
  - Solidity
  - JavaScript
Location:
  - Muhammad
Total Experience:
  - 3 years 8 months


In [34]:
# ------------------------------------------
# ✅ Resume Information Extractor (NER + Email + Experience + Skills + Last Education)
# ------------------------------------------

import spacy
import re
from pdfminer.high_level import extract_text
from dateutil import parser
from datetime import datetime
from spacy.matcher import Matcher

# -----------------------------
# 🔹 Load the trained spaCy NER model
# -----------------------------
nlp = spacy.load("output/model-best")  # Update this path as needed

# -----------------------------
# 🔹 Extract text from PDF resume
# -----------------------------
resume_path = "/content/Muhammad Fareed Full Stack Developer Resume - Muhammad Fareed.pdf"
text = extract_text(resume_path)
doc = nlp(text)

# -----------------------------
# 🔹 Initialize data container
# -----------------------------
extracted_data = {
    "Name": set(),
    "College Name": set(),
    # "Location": set(),
    "Skills": set(),
    "Email": set(),
    "Last Education": set(),
    "Experience": set(),
}

# -----------------------------
# 🔹 NER-Based Entity Extraction
# -----------------------------
for ent in doc.ents:
    label = ent.label_
    if label not in extracted_data:
        extracted_data[label] = set()
    extracted_data[label].add(ent.text.strip())

# -----------------------------
# 🔹 Email Extraction (with constraints)
# -----------------------------
email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"
emails = re.findall(email_pattern, text)
for email in emails:
    if len(email) > 5 and "." in email.split("@")[-1]:
        extracted_data["Email"].add(email.lower())

# -----------------------------
# 🔹 Experience Extraction from date patterns
# -----------------------------
experience_pattern = re.findall(
    r"(.+?)\s*\(?([A-Za-z]+\s+\d{2,4})\)?\s*[–-]\s*\(?([A-Za-z]+\s+\d{2,4}|Present)\)?",
    text
)

raw_date_ranges = []
for job_title, start_str, end_str in experience_pattern:
    try:
        start_date = parser.parse(start_str, default=datetime(1900, 1, 1))
        end_date = datetime.today() if "Present" in end_str else parser.parse(end_str, default=datetime.today())
        raw_date_ranges.append((start_date, end_date))
        extracted_data["Experience"].add(
            f"{job_title.strip()} ({start_date.strftime('%b %Y')} - {end_date.strftime('%b %Y')})"
        )
    except:
        continue

# Merge overlapping experience durations
raw_date_ranges.sort()
merged_ranges = []
for current_start, current_end in raw_date_ranges:
    if not merged_ranges:
        merged_ranges.append((current_start, current_end))
    else:
        last_start, last_end = merged_ranges[-1]
        if current_start <= last_end:
            merged_ranges[-1] = (last_start, max(last_end, current_end))
        else:
            merged_ranges.append((current_start, current_end))

# Calculate total experience duration
total_months = sum((end.year - start.year) * 12 + (end.month - start.month) for start, end in merged_ranges)
years, months = divmod(total_months, 12)
extracted_data["Total Experience"] = {f"{years} years {months} months"}

# -----------------------------
# 🔹 Rule-Based Skill Matching
# -----------------------------
matcher = Matcher(nlp.vocab)
skills = [
    "python", "javascript", "machine learning", "data science", "mern stack",
    "react", "firebase", "fast api", "solidity", "postgresql"
]
for skill in skills:
    pattern = [{"LOWER": token} for token in skill.split()]
    matcher.add("SKILLS", [pattern])

matches = matcher(doc)
for match_id, start, end in matches:
    extracted_data["Skills"].add(doc[start:end].text)

# -----------------------------
# 🔹 Last Education Detection (latest mentioned college/university)
# -----------------------------
education_labels = ["College Name", "University", "Institute", "School", "Education"]
education_entities = []
for ent in doc.ents:
    if ent.label_ in education_labels:
        education_entities.append((ent.start_char, ent.text.strip()))

if education_entities:
    education_entities.sort()  # sort by text position
    last_education = education_entities[-1][1]
    extracted_data["Last Education"] = {last_education}

# -----------------------------
# 🔹 Display Extracted Results
# -----------------------------
print("\n---- 📄 Extracted Resume Information ----")
for key, values in extracted_data.items():
    if values:
        print(f"{key}:\n  - " + "\n  - ".join(sorted(values)))



---- 📄 Extracted Resume Information ----
Name:
  - MUHAMMAD FAREED
College Name:
  - Hamdard University
Skills:
  - FAST API
  - Firebase
  - JavaScript
  - MERN Stack
  - MERN stack
  - PostgreSQL
  - Python
  - React
  - Solidity
Email:
  - mfareed1504@gmail.com
Last Education:
  - Hamdard University
Experience:
  - Frontend Developer / Elearning Avenue (Oct 2022 - May 2023)
  - Instructor & Trainer / Saylani Mass IT (Dec 2023 - Jul 2025)
  - Karachi, Pakistan (Nov 2021 - Jul 2025)
  - MERN Stack Developer / Esspfa IT Solution (Jul 2023 - Jul 2025)
Location:
  - Muhammad
Total Experience:
  - 3 years 8 months


In [35]:
# ------------------------------------------
# ✅ Resume Information Extractor (NER + Email + Experience + Skills + Last Education)
# ------------------------------------------

import spacy
import re
from pdfminer.high_level import extract_text
from dateutil import parser
from datetime import datetime
from spacy.matcher import Matcher

# -----------------------------
# 🔹 Load the trained spaCy NER model
# -----------------------------
nlp = spacy.load("output/model-best")  # Update this path as needed

# -----------------------------
# 🔹 Extract text from PDF resume
# -----------------------------
resume_path = "/content/Muhammad Naeemuddin Resume.pdf"
text = extract_text(resume_path)
doc = nlp(text)

# -----------------------------
# 🔹 Initialize data container
# -----------------------------
extracted_data = {
    "Name": set(),
    "College Name": set(),
    # "Location": set(),
    "Skills": set(),
    "Email": set(),
    "Last Education": set(),
    "Experience": set(),
}

# -----------------------------
# 🔹 NER-Based Entity Extraction
# -----------------------------
for ent in doc.ents:
    label = ent.label_
    if label not in extracted_data:
        extracted_data[label] = set()
    extracted_data[label].add(ent.text.strip())

# -----------------------------
# 🔹 Email Extraction (with constraints)
# -----------------------------
email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"
emails = re.findall(email_pattern, text)
for email in emails:
    if len(email) > 5 and "." in email.split("@")[-1]:
        extracted_data["Email"].add(email.lower())

# -----------------------------
# 🔹 Experience Extraction from date patterns
# -----------------------------
experience_pattern = re.findall(
    r"(.+?)\s*\(?([A-Za-z]+\s+\d{2,4})\)?\s*[–-]\s*\(?([A-Za-z]+\s+\d{2,4}|Present)\)?",
    text
)

raw_date_ranges = []
for job_title, start_str, end_str in experience_pattern:
    try:
        start_date = parser.parse(start_str, default=datetime(1900, 1, 1))
        end_date = datetime.today() if "Present" in end_str else parser.parse(end_str, default=datetime.today())
        raw_date_ranges.append((start_date, end_date))
        extracted_data["Experience"].add(
            f"{job_title.strip()} ({start_date.strftime('%b %Y')} - {end_date.strftime('%b %Y')})"
        )
    except:
        continue

# Merge overlapping experience durations
raw_date_ranges.sort()
merged_ranges = []
for current_start, current_end in raw_date_ranges:
    if not merged_ranges:
        merged_ranges.append((current_start, current_end))
    else:
        last_start, last_end = merged_ranges[-1]
        if current_start <= last_end:
            merged_ranges[-1] = (last_start, max(last_end, current_end))
        else:
            merged_ranges.append((current_start, current_end))

# Calculate total experience duration
total_months = sum((end.year - start.year) * 12 + (end.month - start.month) for start, end in merged_ranges)
years, months = divmod(total_months, 12)
extracted_data["Total Experience"] = {f"{years} years {months} months"}

# -----------------------------
# 🔹 Rule-Based Skill Matching
# -----------------------------
matcher = Matcher(nlp.vocab)
skills = [
    "python", "javascript", "machine learning", "data science", "mern stack",
    "react", "firebase", "fast api", "solidity", "postgresql"
]
for skill in skills:
    pattern = [{"LOWER": token} for token in skill.split()]
    matcher.add("SKILLS", [pattern])

matches = matcher(doc)
for match_id, start, end in matches:
    extracted_data["Skills"].add(doc[start:end].text)

# -----------------------------
# 🔹 Last Education Detection (latest mentioned college/university)
# -----------------------------
education_labels = ["College Name", "University", "Institute", "School", "Education"]
education_entities = []
for ent in doc.ents:
    if ent.label_ in education_labels:
        education_entities.append((ent.start_char, ent.text.strip()))

if education_entities:
    education_entities.sort()  # sort by text position
    last_education = education_entities[-1][1]
    extracted_data["Last Education"] = {last_education}

# -----------------------------
# 🔹 Display Extracted Results
# -----------------------------
print("\n---- 📄 Extracted Resume Information ----")
for key, values in extracted_data.items():
    if values:
        print(f"{key}:\n  - " + "\n  - ".join(sorted(values)))



---- 📄 Extracted Resume Information ----
Name:
  - MUHAMMAD NAEEMUDDIN
College Name:
  - St. Patrick’s High School
Skills:
  - Firebase
  - JavaScript
  - Javascript
  - PostgreSQL
  - Python
Email:
  - naeemuddin24@gmail.com
Last Education:
  - St. Patrick’s High School
Location:
  - Karachi
Email Address:
  - https://github.com/MuhammadNaeemuddin
Total Experience:
  - 0 years 0 months


In [36]:
# ------------------------------------------
# ✅ Resume Information Extractor (NER + Email + Experience + Skills + Last Education)
# ------------------------------------------

import spacy
import re
from pdfminer.high_level import extract_text
from dateutil import parser
from datetime import datetime
from spacy.matcher import Matcher

# -----------------------------
# 🔹 Load the trained spaCy NER model
# -----------------------------
nlp = spacy.load("output/model-best")  # Update this path as needed

# -----------------------------
# 🔹 Extract text from PDF resume
# -----------------------------
resume_path = "/content/Muhammad Raza Resume (1).pdf"
text = extract_text(resume_path)
doc = nlp(text)

# -----------------------------
# 🔹 Initialize data container
# -----------------------------
extracted_data = {
    "Name": set(),
    "College Name": set(),
    # "Location": set(),
    "Skills": set(),
    "Email": set(),
    "Last Education": set(),
    "Experience": set(),
}

# -----------------------------
# 🔹 NER-Based Entity Extraction
# -----------------------------
for ent in doc.ents:
    label = ent.label_
    if label not in extracted_data:
        extracted_data[label] = set()
    extracted_data[label].add(ent.text.strip())

# -----------------------------
# 🔹 Email Extraction (with constraints)
# -----------------------------
email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"
emails = re.findall(email_pattern, text)
for email in emails:
    if len(email) > 5 and "." in email.split("@")[-1]:
        extracted_data["Email"].add(email.lower())

# -----------------------------
# 🔹 Experience Extraction from date patterns
# -----------------------------
experience_pattern = re.findall(
    r"(.+?)\s*\(?([A-Za-z]+\s+\d{2,4})\)?\s*[–-]\s*\(?([A-Za-z]+\s+\d{2,4}|Present)\)?",
    text
)

raw_date_ranges = []
for job_title, start_str, end_str in experience_pattern:
    try:
        start_date = parser.parse(start_str, default=datetime(1900, 1, 1))
        end_date = datetime.today() if "Present" in end_str else parser.parse(end_str, default=datetime.today())
        raw_date_ranges.append((start_date, end_date))
        extracted_data["Experience"].add(
            f"{job_title.strip()} ({start_date.strftime('%b %Y')} - {end_date.strftime('%b %Y')})"
        )
    except:
        continue

# Merge overlapping experience durations
raw_date_ranges.sort()
merged_ranges = []
for current_start, current_end in raw_date_ranges:
    if not merged_ranges:
        merged_ranges.append((current_start, current_end))
    else:
        last_start, last_end = merged_ranges[-1]
        if current_start <= last_end:
            merged_ranges[-1] = (last_start, max(last_end, current_end))
        else:
            merged_ranges.append((current_start, current_end))

# Calculate total experience duration
total_months = sum((end.year - start.year) * 12 + (end.month - start.month) for start, end in merged_ranges)
years, months = divmod(total_months, 12)
extracted_data["Total Experience"] = {f"{years} years {months} months"}

# -----------------------------
# 🔹 Rule-Based Skill Matching
# -----------------------------
matcher = Matcher(nlp.vocab)
skills = [
    "python", "javascript", "machine learning", "data science", "mern stack",
    "react", "firebase", "fast api", "solidity", "postgresql"
]
for skill in skills:
    pattern = [{"LOWER": token} for token in skill.split()]
    matcher.add("SKILLS", [pattern])

matches = matcher(doc)
for match_id, start, end in matches:
    extracted_data["Skills"].add(doc[start:end].text)

# -----------------------------
# 🔹 Last Education Detection (latest mentioned college/university)
# -----------------------------
education_labels = ["College Name", "University", "Institute", "School", "Education"]
education_entities = []
for ent in doc.ents:
    if ent.label_ in education_labels:
        education_entities.append((ent.start_char, ent.text.strip()))

if education_entities:
    education_entities.sort()  # sort by text position
    last_education = education_entities[-1][1]
    extracted_data["Last Education"] = {last_education}

# -----------------------------
# 🔹 Display Extracted Results
# -----------------------------
print("\n---- 📄 Extracted Resume Information ----")
for key, values in extracted_data.items():
    if values:
        print(f"{key}:\n  - " + "\n  - ".join(sorted(values)))



---- 📄 Extracted Resume Information ----
Name:
  - MUHAMMAD RAZA
College Name:
  - SRE-Majeed Govt. Degree College
Intermediate (Pre-Engineering)
Skills:
  - Data Science
  - Machine Learning
  - Python
  - data science
  - machine learning
Email:
  - mrezasyed3@gmail.com
Last Education:
  - SRE-Majeed Govt. Degree College
Intermediate (Pre-Engineering)
Location:
  - Karachi
Total Experience:
  - 0 years 0 months


In [37]:
# ------------------------------------------
# ✅ Resume Information Extractor (NER + Email + Experience + Skills + Last Education)
# ------------------------------------------

import spacy
import re
from pdfminer.high_level import extract_text
from dateutil import parser
from datetime import datetime
from spacy.matcher import Matcher

# -----------------------------
# 🔹 Load the trained spaCy NER model
# -----------------------------
nlp = spacy.load("output/model-best")  # Update this path as needed

# -----------------------------
# 🔹 Extract text from PDF resume
# -----------------------------
resume_path = "/content/Abdullah_CV.pdf"
text = extract_text(resume_path)
doc = nlp(text)

# -----------------------------
# 🔹 Initialize data container
# -----------------------------
extracted_data = {
    "Name": set(),
    "College Name": set(),
    # "Location": set(),
    "Skills": set(),
    "Email": set(),
    "Last Education": set(),
    "Experience": set(),
}

# -----------------------------
# 🔹 NER-Based Entity Extraction
# -----------------------------
for ent in doc.ents:
    label = ent.label_
    if label not in extracted_data:
        extracted_data[label] = set()
    extracted_data[label].add(ent.text.strip())

# -----------------------------
# 🔹 Email Extraction (with constraints)
# -----------------------------
email_pattern = r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b"
emails = re.findall(email_pattern, text)
for email in emails:
    if len(email) > 5 and "." in email.split("@")[-1]:
        extracted_data["Email"].add(email.lower())

# -----------------------------
# 🔹 Experience Extraction from date patterns
# -----------------------------
experience_pattern = re.findall(
    r"(.+?)\s*\(?([A-Za-z]+\s+\d{2,4})\)?\s*[–-]\s*\(?([A-Za-z]+\s+\d{2,4}|Present)\)?",
    text
)

raw_date_ranges = []
for job_title, start_str, end_str in experience_pattern:
    try:
        start_date = parser.parse(start_str, default=datetime(1900, 1, 1))
        end_date = datetime.today() if "Present" in end_str else parser.parse(end_str, default=datetime.today())
        raw_date_ranges.append((start_date, end_date))
        extracted_data["Experience"].add(
            f"{job_title.strip()} ({start_date.strftime('%b %Y')} - {end_date.strftime('%b %Y')})"
        )
    except:
        continue

# Merge overlapping experience durations
raw_date_ranges.sort()
merged_ranges = []
for current_start, current_end in raw_date_ranges:
    if not merged_ranges:
        merged_ranges.append((current_start, current_end))
    else:
        last_start, last_end = merged_ranges[-1]
        if current_start <= last_end:
            merged_ranges[-1] = (last_start, max(last_end, current_end))
        else:
            merged_ranges.append((current_start, current_end))

# Calculate total experience duration
total_months = sum((end.year - start.year) * 12 + (end.month - start.month) for start, end in merged_ranges)
years, months = divmod(total_months, 12)
extracted_data["Total Experience"] = {f"{years} years {months} months"}

# -----------------------------
# 🔹 Rule-Based Skill Matching
# -----------------------------
matcher = Matcher(nlp.vocab)
skills = [
    "python", "javascript", "machine learning", "data science", "mern stack",
    "react", "firebase", "fast api", "solidity", "postgresql"
]
for skill in skills:
    pattern = [{"LOWER": token} for token in skill.split()]
    matcher.add("SKILLS", [pattern])

matches = matcher(doc)
for match_id, start, end in matches:
    extracted_data["Skills"].add(doc[start:end].text)

# -----------------------------
# 🔹 Last Education Detection (latest mentioned college/university)
# -----------------------------
education_labels = ["College Name", "University", "Institute", "School", "Education"]
education_entities = []
for ent in doc.ents:
    if ent.label_ in education_labels:
        education_entities.append((ent.start_char, ent.text.strip()))

if education_entities:
    education_entities.sort()  # sort by text position
    last_education = education_entities[-1][1]
    extracted_data["Last Education"] = {last_education}

# -----------------------------
# 🔹 Display Extracted Results
# -----------------------------
print("\n---- 📄 Extracted Resume Information ----")
for key, values in extracted_data.items():
    if values:
        print(f"{key}:\n  - " + "\n  - ".join(sorted(values)))



---- 📄 Extracted Resume Information ----
Name:
  - MUHAMMAD ABDULLAH
College Name:
  - Government Degree College SRE-Majeed, Karachi

2017/April
2019/July

PROJECTS

Matriculation - Science
-
Computer Science

Happy Palace Grammar School, Karachi

Machine Learning Object Detection

GitHub Link
The model is a real-time image classification system designed to predict the class of objects (e.g., fruits
and vegetables) captured through a live video feed from a webcam. It uses a pre-trained deep learning
model loaded from a .h5 file to classify images into predefined classes.

Power BI

HR Dashboard
A basic analysis of the employees of the company.

Machine Learning Stock Prediction

A basic prediction of stocks, using Linear Regression.

EXPERIENCE

GitHub Link

GitHub Link

Feb 2025 – Present Project Manager Intern

Binate Digital
Assisting in managing software development projects, coordinating with cross-functional teams, and en-
suring timely delivery of project milestones.

CERTIFICA

In [ ]:
# import spacy
# import re
# from pdfminer.high_level import extract_text
# from dateutil import parser
# from datetime import datetime

# # Load the trained NER model
# nlp = spacy.load("output/model-last")

# # Extract text from the resume
# text = extract_text("/content/Muhammad Fareed Full Stack Developer Resume - Muhammad Fareed.pdf")
# doc = nlp(text)

# # Store extracted entities in a dictionary (Use Sets to Avoid Duplicates)
# extracted_data = {
#     "Name": set(),
#     "Degree": set(),
#     "College Name": set(),
#     "Location": set(),
#     "Skills": set(),
#     "Experience": set(),
# }

# # Extract Named Entities (NER)
# for ent in doc.ents:
#     extracted_data[ent.label_].add(ent.text.strip())

# # Remove unwanted terms from Skills & Degree
# invalid_terms = {"SKILLS", "SUMMARY", "WORK EXPERIENCE", "EDUCATION"}
# extracted_data["Degree"] -= invalid_terms
# extracted_data["Skills"] -= invalid_terms

# # Extract skills manually (if missing from NER)
# skills_section = re.findall(
#     r"(?i)(?:Skills|Technical Skills|Expertise|SKILLS SUMMARY)[\s\S]*?(?=\n\n|\Z)", text
# )
# if skills_section:
#     skills = re.findall(r"\b[A-Za-z#.\-+]+\b", skills_section[0])
#     extracted_data["Skills"].update(skill for skill in skills if skill.upper() not in invalid_terms)

# # Extract job experience details (using regex)
# experience_pattern = re.findall(
#     r"([\w\s\-/]+)\s*[\n]*\s*(\w+\s*\d{4})\s*–\s*(\w+\s*\d{4}|Present)", text
# )

# total_experience_months = 0
# for job_title, start_date, end_date in experience_pattern:
#     try:
#         start_date = parser.parse(start_date)
#         end_date = (
#             parser.parse(end_date) if "Present" not in end_date else datetime.today()
#         )
#         experience_months = (end_date.year - start_date.year) * 12 + (
#             end_date.month - start_date.month
#         )
#         total_experience_months += experience_months

#         extracted_data["Experience"].add(
#             f"{job_title.strip()} ({start_date.strftime('%b %Y')} - {end_date.strftime('%b %Y')})"
#         )
#     except Exception:
#         continue  # Skip parsing errors

# # Convert total experience to years & months
# if total_experience_months > 0:
#     years, months = divmod(total_experience_months, 12)
#     extracted_data["Total Experience"] = f"{years} years {months} months"

# # Remove duplicates & format results properly
# for key in extracted_data:
#     extracted_data[key] = list(set(extracted_data[key]))  # Convert sets to lists

# # Display the cleaned-up results
# print("\n---- Extracted Information ----")
# for key, values in extracted_data.items():
#     if values:
#          print(f"{key}: {' '.join(values)}")


In [ ]:
# print("---- Full Resume Text ----")
# print(text)


In [ ]:
# import sys , fitz

In [ ]:
# fname = '/content/CV-Parsing-using-Spacy-3/data/test/Muhammad Raza Resume (1).pdf'
# doc = fitz.open(fname)
# text = ""
# for page in doc:
  # text = text + str(page.get_text())
# doc.close()

In [ ]:
# text = text.strip()

In [ ]:
# txt = " ".join(text.split('\n'))

In [ ]:
# print(txt)

In [ ]:
# doc = nlp(text)
# for ent in doc.ents:
  # print(ent.text, '---->' , ent.label_)

In [39]:
!zip -r /content/output.zip /content/output

  adding: content/output/ (stored 0%)
  adding: content/output/model-last/ (stored 0%)
  adding: content/output/model-last/transformer/ (stored 0%)
  adding: content/output/model-last/transformer/cfg (stored 0%)
  adding: content/output/model-last/transformer/model (deflated 13%)
  adding: content/output/model-last/ner/ (stored 0%)
  adding: content/output/model-last/ner/cfg (deflated 33%)
  adding: content/output/model-last/ner/moves (deflated 74%)
  adding: content/output/model-last/ner/model (deflated 8%)
  adding: content/output/model-last/config.cfg (deflated 61%)
  adding: content/output/model-last/tokenizer (deflated 81%)
  adding: content/output/model-last/vocab/ (stored 0%)
  adding: content/output/model-last/vocab/strings.json (deflated 76%)
  adding: content/output/model-last/vocab/lookups.bin (stored 0%)
  adding: content/output/model-last/vocab/key2row (stored 0%)
  adding: content/output/model-last/vocab/vectors (deflated 45%)
  adding: content/output/model-last/vocab/vec

In [ ]:
# !python -m spacy debug data config.cfg


---
<h4>For Reference:</h4>
<a href='https://github.com/laxmimerit/CV-Parsing-using-Spacy-3.git'>CV-Parsing-using-Spacy-3
</a><br>
<a href='https://github.com/yashlikescode/spacyResumeParcer.git'>spacyResumeParcer</a>
<hr>